# Modèles à variable dépendante limitée

Cette application reprend la proposition de Stock et Watson (le manuel) qui utilisent les données de l'article de Alicia H Munnell, Geoffrey MB Tootell, Lynn E Browne, and James McEneaney : "Mortgage lending in
boston: Interpreting hmda data." *The American Economic Review*, pages 25-53, 1996.

Dans cet article, les chercheurs de la Federal Reserve Bank de Boston souhaitent savoir les noirs américains sont discriminés dans l'accès au crédit immobilier. Ils collectent des données sur les demandes de prêts immobiliers en 1990 dans Boston et sa banlieue. Ils obtiennent 2380 observations de demandeurs blanc et noir-américain. 

Les données sont décrites dans le codebook simplifié : **TD6_codebook.png** : 

![title](TD6_codebook.png)

## Intuition
On ouvre la base de données : **TD6_hdma.dta**. 

In [1]:
library(foreign)
#hdma<-

[1] 2380   14

deny,pi_rat,black,hse_inc,loan_val,ccred,mcred,pubrec,denpmi,selfemp,single,hischl,probunmp,condo
0,0.221,0,0.221,0.8000000,5,2,0,0,0,0,1,3.9,0
0,0.265,0,0.265,0.9218750,2,2,0,0,0,1,1,3.2,0
0,0.372,0,0.248,0.9203980,1,2,0,0,0,0,1,3.2,0
0,0.320,0,0.250,0.8604651,1,2,0,0,0,0,1,4.3,0
0,0.360,0,0.350,0.6000000,1,1,0,0,0,0,1,3.2,0
0,0.240,0,0.170,0.5105263,1,1,0,0,0,0,1,3.9,0


Les noirs-américains (*black*) ont-ils effectivement une probabilité plus importante de voir leur demande de prêt rejetée (*deny*) ? 

*Indice : proposez un test d'égalité de moyennes ou une régression linéaire simple.*

In [ ]:
#test d'égalité de moyennes


In [ ]:
#régression linéaire simple
#lpm1 <- 

Grâce à des tests d'égalité de moyennes, déterminez si les blancs-américains sont de bons contrefactuels pour les noirs-américains. Peut-on donc interpréter la différence de traitement de la question précédente comme de la discrimination ?

In [ ]:
a<-t.test(hdma$selfemp ~ hdma$black)
b<-t.test(hdma$single ~ hdma$black)
c<-t.test(hdma$hischl ~ hdma$black)
d<-t.test(hdma$probunmp ~ hdma$black)
print(c(a$statistic,b$statistic,c$statistic,d$statistic))
print(c(a$p.value,b$p.value,c$p.value,d$p.value))


## 1. Modèle de probabilité linéaire 
On s'intéresse à contrôler en plus pour l'effet du ratio mensualité de l'emprunt / revenu total *pi_rat* sur la probabilité de rejet. Pourquoi ? Que s'attend-on à trouver?

Rappeler ce qu'est un modèle de probabilité linéaire.

Estimer et interpréter.

In [ ]:
summary(hdma$pi_rat)
lpm2 <- lm(deny~black+pi_rat,data=hdma)
summary(lpm2)
#coeftest(lpm2, vcov = vcovHC(lpm2, type="HC1")) 

## 2. Modèle de régression probit

On estime le modèle de régression probit de *deny* sur *black* et *pi_rat* par maximum de vraisemblance : **glm( y~x, binomial("probit"), data=... )**. Quel est le principe de l'estimation par maximum de vraisemblance ? Interpréter les résultats.

In [ ]:
probit1 <- glm(deny ~ black + pi_rat, binomial("probit"), data = hdma)
summary(probit1)

Pourquoi l'interprétation des résultats est-elle compliquée ? 

#### 1. Effet marginal à la moyenne 
Rappeler ce qu'est l'effet marginal à la moyenne.

En vous aidant du code ci-dessous, calculez l'effet marginal du fait d'être noir-américain sur la probabilité de rejet. Pour cela, calculez d'abord la probabilité de rejet de l'individu moyen noir, puis de l'individu moyen blanc. Calculez ensuite la différence.

*Indice: La fonction de répartition de la loi normale s'écrit :* **pnorm()** 

In [ ]:
#voir les coefficients:
coef(probit1)

In [ ]:
#Prob for blacks at means
z1  <- coef(probit1)[1] + coef(probit1)[2] + coef(probit1)[3]*mean(hdma$pi_rat)
Gz1 <- pnorm(z1)
#Prob for whites at means
#z0  <- 
#Gz0 <- 
#Difference in probs
Gz1-Gz0

Calculer l'effet marginal du fait d'avoir un ratio d'endettement de 10 points de pourcentage plus élevé pour l'individu moyen. Pour ce faire calculez la proba de rejet de l'individu moyen, puis de l'individu moyen avec 0.1 de ratio d'endettement supplémentaire. Calculez ensuite la différence.

In [ ]:
hdma$black2<-ifelse(hdma$black=="Black",1,0)   #j'ai besoin que black soit numérique

###Calculer l'effet marginal du fait d'avoir un ratio d'endettement de 10 points de pourcentage plus ?lev? pour l'individu moyen.
#Prob at means
z0  <- coef(probit1)[1] + coef(probit1)[2]*mean(hdma$black2) + coef(probit1)[3]*mean(hdma$pi_rat)
Gz0 <- pnorm(z0)
#z1  <- 
#Gz1 <- 
Gz1-Gz0

#### 2. Effet marginal moyen

Rappeler ce qu'est l'effet marginal moyen.






Que mesure ce code ?

*Indice: La fonction de densité de la loi normale s'écrit :* **dnorm()**

In [ ]:
hdma$gg<-dnorm(coef(probit1)[1] + coef(probit1)[2]*hdma$black2 + coef(probit1)[3]*hdma$pi_rat)
scalefactor<-mean(hdma$gg)
scalefactor

Calculer l'effet marginal moyen du ratio d'endettement. 

In [ ]:
effet_marginal_moyen <-scalefactor*coef(probit1)[3]
effet_marginal_moyen

In [ ]:
#NOTE : on peut calculer l'effet marginal très simplement avec margins :
install.packages("margins")
library(margins)
AME<-margins(probit1)
summary(AME)

## 3. Comparaison des deux modèles de régression

Calculer la probabilité de rejet quand le demandeur est blanc avec un taux d'endettement de 0.1 selon le modèle de probabilité linéaire et le modèle probit. Commentez. 



In [ ]:
#Par MPL
ypred <- coef(lpm2)[1] + coef(lpm2)[3]*0.1
#Par Probit
z <- coef(probit1)[1] + coef(probit1)[3]*0.1
p=pnorm(z)
ypred
p

Que réprésentent les commandes et le graphique ci-dessous ?

In [ ]:
library(ggplot2)
hdma$p_rejet <- predict(probit1, hdma, type = "response")
hdma$y_hat <- predict(lpm2, hdma, type = "response")
ggplot(hdma, aes(x = pi_rat, y = p_rejet, colour = black))+ geom_point()+geom_line(aes(y = y_hat, colour = black))

## 4. Modèle de régression probit complet

Quelles sont les autres variables que vous souhaiteriez ajouter à la régression ? 

Sous quelle condition estimez-vous l'effet causal du traitement ?

Estimer le modèle probit de *black* sur *deny* en ajoutant 1 puis 2 puis 3 catégories de variables : 
- emprunt : pi_rat + hse_inc + loan_val 
- emprunteur : ccred + mcred + pubrec + denpmi
- individuelles : selfemp + single + hischl + probunmp

Afficher directement les effets marginaux en utilisant **summary(margins(** *objet estimé par glm()*)**)**. Interpréter les résultats.

In [ ]:
probit2<-glm(deny ~ black + pi_rat + hse_inc + loan_val  , binomial("probit"), data = hdma)
probit3<-glm(deny ~ black + pi_rat + hse_inc + loan_val + ccred + mcred + pubrec + denpmi  , binomial("probit"), data = hdma)
probit4<-glm(deny ~ black + pi_rat + hse_inc + loan_val + ccred + mcred + pubrec + denpmi + selfemp + single + hischl + probunmp , binomial("probit"), data = hdma)

summary(margins(probit1))
summary(margins(probit2))
summary(margins(probit3))
summary(margins(probit4))

Qu'ajouteriez-vous à la régression pour savoir si le ratio d'endettement a un effet différent sur la probabilité de voir son dossier rejeté par la banque selon si on est noir ou blanc ?

Voici le pseudoR2 de McFadden pour le modèle simple et multiple. Commentez la qualité générale des régressions.

In [ ]:
pR2_1 <- 1 - probit1$deviance / probit1$null.deviance 
pR2_2 <- 1 - probit2$deviance / probit2$null.deviance 
pR2_3 <- 1 - probit3$deviance / probit3$null.deviance 
pR2_4 <- 1 - probit4$deviance / probit4$null.deviance 
pR2_1
pR2_2
pR2_3
pR2_4

<font color=green> 
*****************************************************************
RAPPEL au cas où : Le **pseudo R2** est un équivalent du R2 pour les modèles de régression probit. Il y en a plusieurs. 
    
On peut partir de celui de **McFadden : 1 – Lu/L0** avec:
- Lu log vraisemblance du modèle 
- et L0 log vraisemblance du modèle estimé avec seulement une constante. 

=> Si le modèle avec variables n’a aucun pouvoir explicatif alors Lu sera égal à L0 et donc Le R2 sera égal à 0. 

=> Plus Lu se rapprocge de 0 (rappelons que l’on maximise Lu et que Lu est négative donc sa maximisation implique qu’elle se rapproche de 0), plus le ratio Lu/L0 se rapproche de 0 donc R2 se rapproche de 1. 

Calcul :
- Use deviance since $deviance = -2 \times ln(Lfull)$ and $null.deviance = -2 \times ln(Lnull)$ donc pseudoR2 = 1 - (Lu/L0) = 1 - (deviance/null.deviance)
- ICI: on a l’info sur la deviance (en bas du tableau de l'estimation probit) : pR2 = 1-(1264.9/1726.9)=0.2675. 
*****************************************************************
</font>